In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen
from urllib.parse import quote_plus
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import time
import os
import re

In [2]:
current_path = os.getcwd()
print(print(current_path))

/Users/minjun/Downloads/FooBao_Crawling-main
None


In [3]:
restaurant_name = '마츠도 서울역점'

In [4]:
import os
import datetime
#사진 넣을 폴더 생성

now = time.localtime()
current_time = '%04d.%02d.%02d' %(now.tm_year, now.tm_mon, now.tm_mday)

restaurant_dir_name = restaurant_name+"_"+current_time
img_dir_name = "review_"+restaurant_name+"_"+current_time
menu_dir_name = "menu_"+restaurant_name+"_"+current_time

try:
    os.makedirs(restaurant_dir_name)
except FileExistsError:
    print("이미 식당 폴더 있습니다")
    
os.chdir(restaurant_dir_name)


try:
    os.makedirs(img_dir_name)
    #os.chdir(img_dir_name)
except FileExistsError:
    print("이미 img 폴더 있습니다")
try:
    os.makedirs(menu_dir_name)
    #os.chdir(menu_dir_name)
except FileExistsError:
    print("이미 menu 폴더 있습니다")
print(os.getcwd())

/Users/minjun/Downloads/FooBao_Crawling-main/마츠도 서울역점_2023.12.05


In [12]:
driver_path = '../chromedriver'  #driver 위치에 따라 수정

#마음대로 안꺼지게 하는 옵션
service = Service(executable_path=driver_path)
options = Options()
#options.add_argument("--start-maximized")
#options.add_experimental_option("detach",True)

driver = webdriver.Chrome(service=service, options=options)

url = 'https://m.map.naver.com/p?c=15.00,0,0,0,dh'
driver.get(url)

In [13]:
#상호명 검색
driver.find_element(By.XPATH,'''//*[@id="header"]/header/div[1]/div[2]/div/input''').click()
driver.implicitly_wait(2)
driver.find_element(By.XPATH,'''//*[@id="ct"]/div[1]/div[1]/div/form/input''').send_keys(restaurant_name)
driver.find_element(By.XPATH,'''//*[@id="ct"]/div[1]/div[1]/div/form/input''').send_keys(Keys.RETURN)
driver.implicitly_wait(2)

#클릭
driver.find_element(By.XPATH,'''//*[@id="ct"]/div[2]/ul/li/div[1]/a[2]/div''').click()

In [14]:
#사진 옵션
#가게마다 다름 (홈 소식 메뉴 리뷰 사진 지도 주변 정보) 이런식으로 사진이 5번째에 위치할 경우 XPATH 맨 마지막에 a[5], 4번째 위치할경우 a[4].

driver.find_element(By.XPATH,'''//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[5]''').click()

In [15]:
# "음식"
driver.find_element(By.XPATH,'''//*[@id="app-root"]/div/div/div/div[6]/div[2]/div/div/div/div/span[3]/a''').click()
driver.implicitly_wait(5)

In [16]:
# 리뷰 사진 크롤링
scroll_cnt = 0
max_scroll_cnt = 12    # 사용자 지정 필요

img_dict = {}

while scroll_cnt <= max_scroll_cnt:
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    if True:
        print(f'!----- scroll_cnt = "{scroll_cnt}" -----!')
        
        img_block_list = soup.find_all('div', class_='wzrbN')
        image_ids = [img.find('img')['id'] for img in img_block_list]
        image_urls = [img.find('img')['src'] for img in img_block_list]

        img_dict_temp = {img_id : img_url for img_id, img_url in zip(image_ids, image_urls)}
        img_dict.update(img_dict_temp)
        
    else:
        print(f'No element with scroll_cnt="{scroll_cnt}" found')
        # break

    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(3)
    
    scroll_cnt += 1

#맨 위로 스크롤
for _ in range(10):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_UP)
    time.sleep(0.2)

#############

!----- scroll_cnt = "0" -----!
!----- scroll_cnt = "1" -----!
!----- scroll_cnt = "2" -----!
!----- scroll_cnt = "3" -----!
!----- scroll_cnt = "4" -----!
!----- scroll_cnt = "5" -----!
!----- scroll_cnt = "6" -----!
!----- scroll_cnt = "7" -----!
!----- scroll_cnt = "8" -----!
!----- scroll_cnt = "9" -----!
!----- scroll_cnt = "10" -----!
!----- scroll_cnt = "11" -----!
!----- scroll_cnt = "12" -----!


In [17]:
#메뉴 옵션

#마찬가지로 가게마다 다름 (홈 소식 메뉴 리뷰 사진 지도 주변 정보) 이런식으로 메뉴가 3번째에 위치할 경우 XPATH 맨 마지막에 a[3], 4번째 위치할경우 a[4].
driver.find_element(By.XPATH,'''//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[3]''').click()
driver.implicitly_wait(5)

In [18]:
# 메뉴 사진 크롤링

#더보기 옵션
#맨 밑으로 스크롤 -> 더보기 클릭 -> 스크롤 조금 -> 더보기 클릭 -> ... -> 더보기 안나오면 그제서야 맨위로 스크롤.

#스크롤 다운: "더보기" 버튼을 찾을 수 없을 때까지 반복
while True:
    try:
        # 더보기" 버튼이 나타날 때까지 스크롤 다운
        for _ in range(5):
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(0.2)  # 스크롤 사이의 소폭 대기 시간
        
        #"더보기" 버튼 클릭
        driver.find_element(By.XPATH,'''//*[@id="app-root"]/div/div/div/div[6]/div/div[1]/div[2]/div/a''').click()
        #클릭 후에 페이지 요소가 업데이트 될 때까지 잠시 대기
        time.sleep(0.5)
    
    except NoSuchElementException:
        #"더보기" 버튼이 없을 경우 루프 탈출
        print("더 이상 더보기 버튼이 없습니다.")
        break

#맨 위로 스크롤
for _ in range(10):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_UP)
    time.sleep(0.2)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
menu_items = soup.find_all('div',class_='YBmM2')
time.sleep(2)

driver.quit() #드라이버 종료

더 이상 더보기 버튼이 없습니다.


In [46]:
for i in range(len(menu_items)):
    print(menu_items[i].find(class_='place_blind'))
    print(menu_items[i].find(class_="K0PDV")['style'])
    print()

pattern = re.compile(r'url\("([^"]+)"\)')

menu_dict={}

for item in menu_items:
    #메뉴 이름 추철
    name_span = item.find(class_='place_blind')
    menu_name = name_span.get_text() if name_span else "Unknown"

    #이미지 URL을 추출
    img_style = item.find(class_="K0PDV")
    if img_style and 'style' in img_style.attrs:  #img_style이 존재하고 style 속성이 있는지 확인
        style_attr = img_style['style']
        match = pattern.search(style_attr)
        if match:  # URL 패턴이 매치되면 추출
            img_url = match.group(1)
            img_url = img_url.replace('&amp;', '&')  # '&amp;'를 '&'로 치환
            menu_dict[menu_name] = img_url
        else:
            menu_dict[menu_name] = -1
    else:
        menu_dict[menu_name] = -1

<span class="place_blind">마츠도 미소라멘</span>
width: 100px; height: 100px; background-image: url("https://search.pstatic.net/common/?autoRotate=true&quality=95&type=f320_320&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230426_251%2F1682476903528eOux9_JPEG%2FIMG_7377.jpeg");

<span class="place_blind">마츠도 카라 미소라멘</span>
width: 100px; height: 100px; background-image: url("https://search.pstatic.net/common/?autoRotate=true&quality=95&type=f320_320&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230426_127%2F1682482685422GwDY9_JPEG%2FIMG_7372.jpeg");

<span class="place_blind">마파두부면</span>
width: 100px; height: 100px; background-image: url("https://search.pstatic.net/common/?autoRotate=true&quality=95&type=f320_320&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230426_14%2F1682482719024T4gNx_JPEG%2FIMG_7385.jpeg");

<span class="place_blind">미소아부라소바</span>
width: 100px; height: 100px; background-image: url("https://search.pstatic.net/common/?autoRotate=true&quality=95&type=f320_320&src=https%3A%2F%2Fl

In [47]:
menu_dict

{'마츠도 미소라멘': 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=f320_320&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230426_251%2F1682476903528eOux9_JPEG%2FIMG_7377.jpeg',
 '마츠도 카라 미소라멘': 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=f320_320&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230426_127%2F1682482685422GwDY9_JPEG%2FIMG_7372.jpeg',
 '마파두부면': 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=f320_320&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230426_14%2F1682482719024T4gNx_JPEG%2FIMG_7385.jpeg',
 '미소아부라소바': 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=f320_320&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230426_43%2F1682482757036RsnlK_JPEG%2FIMG_7380.jpeg',
 '카라 미소아부라소바': 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=f320_320&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230717_280%2F1689540649756RxkqH_JPEG%2FIMG_7380.jpeg',
 '에비 아부라소바': 'https://search.pstatic.net/common/?autoRot

In [48]:
import urllib.request

os.chdir(img_dir_name)

save_cnt = 1
for key in img_dict:
    #urllib.request.urlretrieve(img_dict[key], str(restaurant_name)+"_"+str(key)+".jpg")
    urllib.request.urlretrieve(img_dict[key], str(save_cnt)+".jpg")
    if (save_cnt % 10) == 0:
        print("%d 번째 이미지까지 저장 완료" % (save_cnt))
    save_cnt += 1

print("%d 개의 이미지 저장 완료!" % (save_cnt-1))

10 번째 이미지까지 저장 완료
20 번째 이미지까지 저장 완료
30 번째 이미지까지 저장 완료
40 번째 이미지까지 저장 완료
50 번째 이미지까지 저장 완료
60 번째 이미지까지 저장 완료
70 번째 이미지까지 저장 완료
80 번째 이미지까지 저장 완료
90 번째 이미지까지 저장 완료
100 번째 이미지까지 저장 완료
110 번째 이미지까지 저장 완료
120 번째 이미지까지 저장 완료
130 번째 이미지까지 저장 완료
140 번째 이미지까지 저장 완료
150 번째 이미지까지 저장 완료
160 번째 이미지까지 저장 완료
170 번째 이미지까지 저장 완료
180 번째 이미지까지 저장 완료
190 번째 이미지까지 저장 완료
200 번째 이미지까지 저장 완료
210 번째 이미지까지 저장 완료
220 번째 이미지까지 저장 완료
230 번째 이미지까지 저장 완료
240 번째 이미지까지 저장 완료
250 번째 이미지까지 저장 완료
260 번째 이미지까지 저장 완료
270 번째 이미지까지 저장 완료
280 번째 이미지까지 저장 완료
290 번째 이미지까지 저장 완료
300 번째 이미지까지 저장 완료
310 번째 이미지까지 저장 완료
320 번째 이미지까지 저장 완료
320 개의 이미지 저장 완료!


In [49]:
os.chdir("../"+menu_dir_name)

menu_df = pd.DataFrame(list(menu_dict.items()), columns=['Menu Name', 'Image URL'])
menu_df.to_csv('menu_images.csv', index=False)
print(menu_df.head())

save_cnt = 1
for key in menu_dict:
    urllib.request.urlretrieve(menu_dict[key],str(key)+".jpg")
    #urllib.request.urlretrieve(menu_dict[key], str(save_cnt)+".jpg")
    if (save_cnt % 10) == 0:
        print("%d 번째 이미지까지 저장 완료" % (save_cnt))
    save_cnt += 1

print("%d 개의 이미지 저장 완료!" % (save_cnt-1))

     Menu Name                                          Image URL
0     마츠도 미소라멘  https://search.pstatic.net/common/?autoRotate=...
1  마츠도 카라 미소라멘  https://search.pstatic.net/common/?autoRotate=...
2        마파두부면  https://search.pstatic.net/common/?autoRotate=...
3      미소아부라소바  https://search.pstatic.net/common/?autoRotate=...
4   카라 미소아부라소바  https://search.pstatic.net/common/?autoRotate=...
10 번째 이미지까지 저장 완료
17 개의 이미지 저장 완료!
